In [32]:
import psycopg2
import pandas as pd

In [33]:
pip install psycopg2


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
def create_database():
    conn = psycopg2.connect(host="127.0.0.1", dbname="test_db", user="root", password="root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    cur.execute("CREATE DATABASE world")
    conn.close()

    conn = psycopg2.connect(host="127.0.0.1", dbname="world", user="root", password="root")
    cur = conn.cursor()
    
    return cur,conn

In [40]:
cur,conn = create_database()

In [14]:
countries = pd.read_csv("./dataset/countries.csv")

In [15]:
countries.head(1)

,id,name,iso3,iso2,numeric_code,phone_code,capital,currency,currency_name,currency_symbol,...,region,region_id,subregion,subregion_id,nationality,timezones,latitude,longitude,emoji,emojiU
0,1,Afghanistan,AFG,AF,4,93,Kabul,AFN,Afghan afghani,؋,...,Asia,3.0,Southern Asia,14.0,Afghan,"[{zoneName:'Asia\/Kabul',gmtOffset:16200,gmtOf...",33.0,65.0,🇦🇫,U+1F1E6 U+1F1EB


In [16]:
states = pd.read_csv("./dataset/states.csv")

In [17]:
cities = pd.read_csv("./dataset/cities.csv")

In [11]:
states.head()

,name,country_id,country_code,country_name,state_code,type,latitude,longitude
id,,,,,,,,
3901,Badakhshan,1,AF,Afghanistan,BDS,NaN,36.734772,70.811995
3871,Badghis,1,AF,Afghanistan,BDG,NaN,35.167134,63.769538
3875,Baghlan,1,AF,Afghanistan,BGL,NaN,36.178903,68.745306
3884,Balkh,1,AF,Afghanistan,BAL,NaN,36.755060,66.897537
3872,Bamyan,1,AF,Afghanistan,BAM,NaN,34.810007,67.821210


In [12]:
cities.head()

,name,state_id,state_code,state_name,country_id,country_code,country_name,latitude,longitude,wikiDataId
id,,,,,,,,,,
52,Ashkāsham,3901,BDS,Badakhshan,1,AF,Afghanistan,36.68333,71.53333,Q4805192
68,Fayzabad,3901,BDS,Badakhshan,1,AF,Afghanistan,37.11664,70.58002,Q156558
78,Jurm,3901,BDS,Badakhshan,1,AF,Afghanistan,36.86477,70.83421,Q10308323
84,Khandūd,3901,BDS,Badakhshan,1,AF,Afghanistan,36.95127,72.31800,Q3290334
115,Rāghistān,3901,BDS,Badakhshan,1,AF,Afghanistan,37.66079,70.67346,Q2670909


In [18]:
countries.columns

Index(['id', 'name', 'iso3', 'iso2', 'numeric_code', 'phone_code', 'capital',
       'currency', 'currency_name', 'currency_symbol', 'tld', 'native',
       'region', 'region_id', 'subregion', 'subregion_id', 'nationality',
       'timezones', 'latitude', 'longitude', 'emoji', 'emojiU'],
      dtype='object')

In [19]:
countries = countries[['id', 'name','phone_code', 'region','nationality']]

In [20]:
countries.head()

,id,name,phone_code,region,nationality
0,1,Afghanistan,93,Asia,Afghan
1,2,Aland Islands,+358-18,Europe,Aland Island
2,3,Albania,355,Europe,Albanian
3,4,Algeria,213,Africa,Algerian
4,5,American Samoa,+1-684,Oceania,American Samoan


In [22]:
states.columns

Index(['id', 'name', 'country_id', 'country_code', 'country_name',
       'state_code', 'type', 'latitude', 'longitude'],
      dtype='object')

In [23]:
states = states[['id', 'name', 'country_id', 'country_code','state_code', 'type']]

In [24]:
cities.columns

Index(['id', 'name', 'state_id', 'state_code', 'state_name', 'country_id',
       'country_code', 'country_name', 'latitude', 'longitude', 'wikiDataId'],
      dtype='object')

In [25]:
cities=cities[['id', 'name', 'state_id', 'country_id','latitude', 'longitude']]

In [45]:
countries_create_table=("""CREATE TABLE IF NOT EXists countries(
    id int primary key,
    name varchar,
    phone_code varchar,
    region varchar,
    nationality varchar)""")

In [2]:
state_create_table=("""CREATE TABLE IF NOT EXists state(
    id int primary key,
    name varchar,
    country_id int,
    country_code varchar,
    state_code varchar,
    type varchar,
    CONSTRAINT fk_countries
      FOREIGN KEY(country_id) 
	  REFERENCES countries(id))""")

In [1]:
cities_create_table=("""CREATE TABLE IF NOT EXists cities(
    id int primary key,
    name varchar,
    state_id int,
    country_id int,
    lattitude varchar,
    longitude varchar,
    CONSTRAINT fk_countries
      FOREIGN KEY(country_id) 
	  REFERENCES countries(id),
   CONSTRAINT fk_state
      FOREIGN KEY(state_id) 
	  REFERENCES state(id))""")

In [76]:
conn = psycopg2.connect(host="127.0.0.1", dbname="world", user="root", password="root")
cur = conn.cursor()

In [77]:
cur.execute(countries_create_table)
conn.commit()

In [78]:
cur1 = conn.cursor()
cur1.execute(state_create_table)
conn.commit()

In [79]:

cur.execute(cities_create_table)
conn.commit()

In [60]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error : Could not get cursor to DB")
    print(e)

In [61]:
conn.set_session(autocommit=True)

ProgrammingError: set_session cannot be used inside a transaction

In [62]:
# try:
#     cur.execute("create database myfirstdb")
# except psycopg2.Error as e:
#     print("Error : Could not create DB")
#     print(e)
    

In [63]:
try:
    cur.execute("insert into students(student_id, name, age, gender,subject,marks) values (%s,%s,%s,%s,%s,%s)",(1,"Leeba",23,"F","Maths",100))
except psycopg2.Error as e:
    print("Error : Could not get cursor to DB")
    print(e)

Error : Could not get cursor to DB
current transaction is aborted, commands ignored until end of transaction block



In [15]:
try:
    cur.execute("insert into students(student_id, name, age, gender,subject,marks) values (%s,%s,%s,%s,%s,%s)",(1,"Leeba",23,"F","Maths",100))
except psycopg2.Error as e:
    print("Error : Could not get cursor to DB")
    print(e)